In [ ]:
import sys
import pdb
import os, glob
import numpy as np

from keras.preprocessing import image
from keras.applications import vgg19
from keras.applications import resnet50
from keras.layers import Input, Flatten, Dense, Dropout, Activation, core
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras import backend as K
from keras.engine import Layer
import pandas as pd
import cv2
import pickle
import scipy.io as sio
from sklearn.cluster import KMeans
from sklearn import metrics, decomposition, mixture
from sklearn.manifold import TSNE
import tensorflow as tf
from importlib import reload
import csv
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
%matplotlib inline

#os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [ ]:
def reverse_gradient(X, hp_lambda):# where hp_lambda is the constant which multiplies the flipped gradient.
    '''Flips the sign of the incoming gradient during training.'''
    try:
        reverse_gradient.num_calls += 1
    except AttributeError:
        reverse_gradient.num_calls = 1

    grad_name = "GradientReversal%d" % reverse_gradient.num_calls

    @tf.RegisterGradient(grad_name)
    def _flip_gradients(op, grad):
        return [tf.negative(grad) * hp_lambda]

    g = K.get_session().graph
    with g.gradient_override_map({'Identity': grad_name}):
        y = tf.identity(X)

    return y

class GradientReversal(Layer):
    '''Flip the sign of gradient during training.'''
    def __init__(self, hp_lambda, **kwargs):
        super(GradientReversal, self).__init__(**kwargs)
        self.supports_masking = False
        
        #self.hp_lambda = hp_lambda
        self.hp_lambda = K.variable(hp_lambda,name='hp_lambda')
        
    def build(self, input_shape):
        self.trainable_weights = []
        #super(GradientReversal, self).build(input_shape)

    def call(self, x, mask=None):
        return reverse_gradient(x, self.hp_lambda)

    def get_output_shape_for(self, input_shape):
        return input_shape

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  'hp_lambda': K.get_value(self.hp_lambda)}
        base_config = super(GradientReversal, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
def icc(Y, icc_type='icc2'):
    ''' Calculate intraclass correlation coefficient for data within
        Brain_Data class

    ICC Formulas are based on:
    Shrout, P. E., & Fleiss, J. L. (1979). Intraclass correlations: uses in
    assessing rater reliability. Psychological bulletin, 86(2), 420.

    icc1:  x_ij = mu + beta_j + w_ij
    icc2/3:  x_ij = mu + alpha_i + beta_j + (ab)_ij + epsilon_ij

    Code modifed from nipype algorithms.icc
    https://github.com/nipy/nipype/blob/master/nipype/algorithms/icc.py

    Args:
        icc_type: type of icc to calculate (icc: voxel random effect,
                icc2: voxel and column random effect, icc3: voxel and
                column fixed effect)

    Returns:
        ICC: intraclass correlation coefficient

    '''
    #Y = self.data.T
    [n, k] = Y.shape

    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    # create the design matrix for the different levels
    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                         X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc / n

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        # ICC = (MSR - MSRW) / (MSR + (k-1) * MSRW)
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'icc2':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'icc3':
        # ICC(3,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error)
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)
        
    return ICC

def cartridgeAverage(features,scans,labels):
    nb_scans = len(np.unique(scans))
    nb_labels = len(np.unique(labels))
    dim_features = features.shape[1]
    # scans and labels should be 0,0,0...1,1,1,1,...,2,2,.... 
    for i in range(len(np.unique(scans))):
        scans[scans==np.unique(scans)[i]]=i
    for i in range(len(np.unique(labels))):
        labels[labels==np.unique(labels)[i]]=i
    # Average the features within each cartridge.
    average_cart_features = np.zeros((nb_scans,nb_labels,dim_features))
    nb_slices_per_cart = np.zeros((nb_scans,nb_labels,dim_features))
    for i in range(features.shape[0]):
        average_cart_features[scans[i],labels[i]]+=features[i]
        nb_slices_per_cart[scans[i],labels[i]]+=1
    average_cart_features /= nb_slices_per_cart
    
    return average_cart_features

def clusterFeatures(features,labels, **options):
    nbClusters = len(np.unique(labels))
    # Cluster
    kmeans = KMeans(n_clusters=nbClusters, random_state=0).fit(features)
    results = metrics.homogeneity_completeness_v_measure(labels, kmeans.labels_)
    print('cluster KMEANS:',results)
    
    return results

def clusterGMMFeatures(features,labels, **options):
    nbClusters = len(np.unique(labels))
    # Cluster (covariance type: ['spherical', 'tied', 'diag', 'full'])
    gmm = mixture.GaussianMixture(n_components=nbClusters, covariance_type='spherical').fit(features)
    
    GMMresults = metrics.homogeneity_completeness_v_measure(labels, gmm.predict(features))
    print('cluster GMM:',GMMresults)
    print('covariances:',gmm.covariances_)
    
    return [GMMresults, gmm.covariances_]

def correlateSize(features,scans,labels):
    nScans = len(np.unique(scans))
    nLabels = len(np.unique(labels))
    dim_features = features.shape[1]
    features = np.reshape(features,(nScans,nLabels,dim_features))
    size = np.asarray([164,114,102,82,82,82,108,82,82,77,77,82,148,153,128,128,82])
    size = size[np.unique(scans)]
    pearsons=[]
    # scans and labels should be 0,0,0...1,1,1,1,...,2,2,.... 
    for i in range(len(np.unique(scans))):
        scans[scans==np.unique(scans)[i]]=i
    for i in range(len(np.unique(labels))):
        labels[labels==np.unique(labels)[i]]=i
    # Then calculate the correlation with the size of the image
    nbnan=0
    for iFeat in range(dim_features):
        for iLabels in range(nLabels):
            feat = np.asarray(features[:,iLabels,iFeat])
            pearson = np.corrcoef(feat,size)[0,1]
            if ~np.isnan(pearson):
                pearsons.append(pearson)
            else:
                nbnan+=1
    pearsons=np.asarray(pearsons)
    print('pearson average:',np.average(np.abs(pearsons)),'(nb nan:',nbnan,')')
    
    return np.average(np.abs(pearsons))

def iccAverage(features,nLabels,nScans,type_icc='icc2'):
    features = np.reshape(features,(nScans,nLabels,features.shape[1]))
    features = features.swapaxes(0,1)
    iccs = []
    nbnan=0
    for i in range(features.shape[2]):
        icc1 = icc(features[:,:,i],type_icc)
        if not np.isnan(icc1):
            iccs.append(icc1)
        else:
            nbnan+=1
    print('ICC: ',np.average(iccs),'(nan',nbnan,')')
    
    return np.average(iccs)

def normalize(x):
    av = np.mean(x,axis=0)
    stdv = np.std(x,axis=0)
    
    return (x-av)/stdv

In [ ]:
# Define all parameters
try:
    pathSlices = './data/array_slices.mat'
    pathLabels = './data/array_labels.mat'
    pathScan = './data/array_scan.mat'
except:
    print('mat files to place in ./data/')
# min and max slices values for normalizing
minAllSlices = -1409
maxAllSlices = 747
# Total number classes and scans
nbClass = 10
nbScans = 17
# Number of training classes and scans
nTrainLabels = 5
nTrainScans = 8 # 17 or 8 in the paper
type_icc = 'icc2'
# Different flags
is_domainAdv=False
is_testTrain=True # to evaluate on the train data
is_addLayer = True # To add a layer before the prediction layer
is_scratch = False # To train from scratch (otherwise load imagenet weights)
is_freeze = True # To freeze all but final layers (not to use if training from scratch)
is_tsne = False # To plot tsne 
is_pcaVis = False
is_dataAugm=False # Not finished implement

# Network to use
network = 'VGG' # VGG resnet50
# input size of the network
inSize = 224
# lambda value in the gradient reversal for domain adversarial (1: adversarial. 0: No influence on the main task. -1: The entire network learns the domain)
lbd = 1
# Batch size
batch_size = 32
# Learning rate
l_r=1e-4
# Number of PCA components
nPCA = 4
# Number of epochs
totalEpochs = 2#100
# Number of repetitions
nbRun = 1#100
# For CAM, not used now
target_class = 0
if is_scratch:
    is_freeze = False
    
if not os.path.exists('results'):
    os.makedirs('results')

In [ ]:
array_slices = sio.loadmat(pathSlices)['array_slices'][0]
array_labels = sio.loadmat(pathLabels)['array_labels'][0]
array_scans = sio.loadmat(pathScan)['array_scan'][0]

x_all=list()
# Resize and preprocess images:
for x in array_slices:
    x = cv2.resize(x, (inSize, inSize),interpolation = cv2.INTER_LINEAR) 
    x = 255*(x-minAllSlices)/(maxAllSlices-minAllSlices)
    x = np.asarray(np.dstack((x, x, x)), dtype=np.float64)
    if network=='VGG':
        x_all.append(vgg19.preprocess_input(x))
    elif network=='resnet50':
        x_all.append(resnet50.preprocess_input(x))
x_all=np.asarray(x_all)
y_all=array_labels

# If we train on all scan/all labels, we also test on all scan/all labels as there is no test left.
if nTrainLabels==10:
    nTestLabels = nbClass
else:
    nTestLabels = nbClass-nTrainLabels
if nTrainScans==17:
    nTestScans = nbScans
else:
    nTestScans = nbScans-nTrainScans

dicResults = {}
dicResults['icc_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['icc_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_homogeneity_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_homogeneity_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_completeness_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_completeness_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_v_measure_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['kmeans_v_measure_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_homogeneity_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_homogeneity_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_completeness_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_completeness_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_v_measure_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['GMM_v_measure_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['corrSize_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['corrSize_pca_fc1'] = np.zeros((nbRun,totalEpochs+1))
dicResults['acc'] = np.zeros((nbRun,totalEpochs+1))
if not is_addLayer:
    dicResults['icc_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['icc_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['icc_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['icc_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_homogeneity_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_homogeneity_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_completeness_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_completeness_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_v_measure_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['kmeans_v_measure_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_homogeneity_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_homogeneity_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_completeness_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_completeness_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_v_measure_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['GMM_v_measure_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['corrSize_fc2'] = np.zeros((nbRun,totalEpochs+1))
    dicResults['corrSize_pca_fc2'] = np.zeros((nbRun,totalEpochs+1))
if is_domainAdv:
    dicResults['acc_domain'] = np.zeros((nbRun,totalEpochs+1))
    
clusterResults = np.zeros((nbRun,2,2,3)) # 2 for fc1 and fc2, 2 for with and without PCA, 3 for clustering homogeneity, completeness and v_measure
clusterResultsFinetune = np.zeros((nbRun,totalEpochs,2,2,3))
clusterGMMResults = np.zeros((nbRun,2,2,3))
clusterGMMResultsFinetune = np.zeros((nbRun,totalEpochs,2,2,3))
clusterGMMCovResults = np.zeros((nbRun,2,2,nTestLabels))
clusterGMMCovResultsFinetune = np.zeros((nbRun,totalEpochs,2,2,nTestLabels))
corrResults = np.zeros((nbRun,2,2)) # 2 because with and without pca
corrResultsFinetune = np.zeros((nbRun,totalEpochs,2,2))
corrFeatResults = np.zeros((nbRun,2,2)) # 2 because with and without pca
corrFeatResultsFinetune = np.zeros((nbRun,totalEpochs,2,2))
iccResults = np.zeros((nbRun,2,2)) # 2 because with and without pca
iccResultsFinetune = np.zeros((nbRun,totalEpochs,2,2))
accuracyFinetune = np.zeros((nbRun,totalEpochs))
accuracyFinetune2 = np.zeros((nbRun,totalEpochs))

for iRun in range(nbRun):        
    x_train=list()
    y_train=list()

    # Free the memory and redefine the model to not run out of memory
    K.clear_session()
    if 'model' in locals():
        del model

    if network=='VGG':
        if is_scratch:
            base_model = vgg19.VGG19(weights=None, include_top=True)
        else:
            base_model = vgg19.VGG19(weights='imagenet', include_top=True)
        if is_addLayer:
            x = Dense(100, activation='relu', name='fc3')(base_model.layers[-2].output)
            x = Dropout(0.5)(x)
            if is_domainAdv:
                gr = GradientReversal(hp_lambda=lbd)
                x2 = gr(x)
                x2 = Dense(100, activation='relu', name='fc3b')(x2)
                x2 = Dropout(0.5)(x2)
                x2 = Dense(nTrainScans, activation='softmax', name='predictions2')(x2)
            x = Dense(nTrainLabels, activation='softmax', name='predictions')(x)
        else:
            x = Dense(nTrainLabels, activation='softmax', name='predictions')(base_model.layers[-2].output)
            if is_domainAdv:
                # 2 losses
                gr = GradientReversal(hp_lambda=1)
                x2 = gr(base_model.layers[-2].output)
                x2 = Dense(nTrainScans, activation='softmax', name='predictions2')(x2)

    elif network=='resnet50':
        if is_scratch:
            base_model = resnet50.ResNet50(weights=None, include_top=True)
        else:
            base_model = resnet50.ResNet50(weights='imagenet', include_top=True)
        if is_addLayer:
            x = Dense(100, activation='relu', name='fc1')(base_model.layers[-2].output)
            x = Dropout(0.5)(x)
            if is_domainAdv:
                gr = GradientReversal(hp_lambda=lbd)
                x2 = gr(x)
                x2 = Dense(100, activation='relu', name='fc1c')(x2)
                x2 = Dropout(0.5)(x2)
                x2 = Dense(nTrainScans, activation='softmax', name='predictions2')(x2)

            x = Dense(nTrainLabels, activation='softmax', name='predictions')(x)
        else:
            x = Dense(nTrainLabels, activation='softmax', name='predictions')(base_model.layers[-2].output)
            if is_domainAdv:
                # 2 losses
                gr = GradientReversal(hp_lambda=1)
                x2 = gr(base_model.layers[-2].output)
                x2 = Dense(nTrainScans, activation='softmax', name='predictions2')(x2)
    #Then create the corresponding model 
    if is_domainAdv:
        model = Model(input=base_model.input, outputs=[x,x2])
        if network=='VGG':
            list_nofreeze=['fc3','fc3b','predictions','predictions2']
        elif network=='resnet50':
            list_nofreeze=['fc1','fc1b','predictions','predictions2']
    else:
        model = Model(input=base_model.input, output=x)   
        if network=='VGG':
            list_nofreeze=['fc3','predictions']
        elif network=='resnet50':
            list_nofreeze=['fc1','predictions']

    if is_freeze:
        for layer in model.layers:
            if layer.name not in list_nofreeze:
                layer.trainable=False

    # Randomly keep some labels and scans for training (test clustering on the rest)
    listLabels = list(range(nbClass))
    listScans = list(range(nbScans))
    np.random.seed(iRun) 
    np.random.shuffle(listLabels)
    trainLabels=sorted(listLabels[0:nTrainLabels])
    np.random.shuffle(listScans)
    trainScans=sorted(listScans[0:nTrainScans])
    print('iRun:',iRun,'****************** training labels:',trainLabels,'scans',trainScans,'********************')
    # Create the train set:
    idxTrainLabels = np.zeros(y_all.shape, dtype=bool)
    idxTrainScans = np.zeros(y_all.shape, dtype=bool)
    for n in trainLabels:
        idxTrainLabels |= (y_all==n)
    for n in trainScans:
        idxTrainScans |= (array_scans==n)
    idxTrain = idxTrainLabels & idxTrainScans
    x_train = x_all[idxTrain]
    y_train = y_all[idxTrain]
    # Create the test set:
    if is_testTrain:
        idxTest=idxTrain
        nTestLabels=nTrainLabels
        nTestScans=nTrainScans
    else:
        idxTest = np.ones(y_all.shape, dtype=bool)
        if not nTrainLabels==10:
            for n in trainLabels:
                idxTest &= (y_all!=n)
        if not nTrainScans==17:
            for n in trainScans:
                idxTest &= (array_scans!=n)                

    # Minimize the number of output labels (0001112222... instead of 0003334444...):
    for i in range(len(np.unique(y_train))):
        y_train[y_train==np.unique(y_train)[i]]=i

    # Convert labels to categorical one-hot encoding
    y_train = to_categorical(y_train, num_classes=nTrainLabels)
    if is_domainAdv:
        y_trainDomain = to_categorical(array_scans[idxTrain], num_classes=nTrainScans)

    # Compile model             
    #decay_rate = .1 / totalEpochs
    decay_rate = 0.
    adam = optimizers.Adam(lr=l_r, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=decay_rate)
    sgd = optimizers.SGD(lr=l_r, momentum=0.9,decay=decay_rate)
    if is_domainAdv:
        # Use K.variables for the loss weights to be able to modify them during training
        lw1 = K.variable(1.)
        lw2 = K.variable(1.)            
        model.compile(loss='categorical_crossentropy', optimizer=adam,
                  metrics=['accuracy'],loss_weights=[lw1, lw2])
    else:
        model.compile(loss='categorical_crossentropy', optimizer=adam,
                  metrics=['accuracy'])

        #visualize_class_activation_map(model,(x_all[idxTest])[0],target_class)    
    print('Results without finetuning:')
    
    #Cluster the features before finetuning:    
    if network=='VGG':  
        if is_addLayer:
            intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('fc3').output])
            fc1_features = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))
        else:
            intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('fc1').output,model.get_layer('fc2').output])
            [fc1_features,fc2_features] = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))
    elif network=='resnet50':
        if is_addLayer:
            intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('fc1').output])            
        else:
            intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('flatten_1').output])
        fc1_features = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))  
        fc1_featuresTrain = np.asarray(intermediate_layer_model.predict(x_all[idxTrain]))

    # normalize 
    fc1_features_noAv = normalize(fc1_features)
    
    # Remove NaN
    fc1_features_noAv = fc1_features_noAv[:,~np.isnan(fc1_features_noAv).any(axis=0)]
    print('fc1_features_noAv.shape',fc1_features_noAv.shape)
    # PCA before averaging
    fc1_featuresPCA_noAv = decomposition.PCA(n_components = nPCA).fit_transform(fc1_features_noAv)

    # Average features within cartridge:
    fc1_features = cartridgeAverage(fc1_features,array_scans[idxTest],y_all[idxTest])
    if 'fc2_features' in locals():
        fc2_features = cartridgeAverage(fc2_features,array_scans[idxTest],y_all[idxTest])
    shapeFeat = fc1_features.shape
    labelsAv = np.zeros(shapeFeat[0]*shapeFeat[1],dtype=np.int)
    scansAv = np.zeros(shapeFeat[0]*shapeFeat[1],dtype=np.int)
    i=0
    for i0 in range(shapeFeat[0]):
        for i1 in range(shapeFeat[1]):
            labelsAv[i]=i1
            scansAv[i]=i0
            i+=1
    fc1_features = np.reshape(fc1_features,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))
    if 'fc2_features' in locals():
        fc2_features = np.reshape(fc2_features,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))

    # Normalize 
    fc1_features = normalize(fc1_features)

    if 'fc2_features' in locals():
        av = np.mean(fc2_features,axis=0)
        stdv = np.std(fc2_features,axis=0)
        fc2_features = (fc2_features-av)/stdv

    # Remove NaN
    fc1_features = fc1_features[:,~np.isnan(fc1_features).any(axis=0)]
    if 'fc2_features' in locals():
        fc2_features = fc2_features[:,~np.isnan(fc2_features).any(axis=0)]

    fc1_featuresPCA = decomposition.PCA(n_components = nPCA).fit_transform(fc1_features)

    if 'fc2_features' in locals():
        fc2_featuresPCA = decomposition.PCA(n_components = nPCA).fit_transform(fc2_features) 

    #Intraclass Correlation Coefficient (ICC)
    dicResults['icc_fc1'][iRun,0] = iccAverage(fc1_features,nTestLabels,nTestScans,type_icc)
    dicResults['icc_pca_fc1'][iRun,0] = iccAverage(fc1_featuresPCA,nTestLabels,nTestScans,type_icc)
    if 'fc2_features' in locals():
        dicResults['icc_fc2'][iRun,0] = iccAverage(fc2_features,nTestLabels,nTestScans,type_icc)
        dicResults['icc_pca_fc2'][iRun,0] = iccAverage(fc2_featuresPCA,nTestLabels,nTestScans,type_icc) 

    # KMEANS clustering
    clusterResults = clusterFeatures(fc1_features,labelsAv, norm = "normal")
    dicResults['kmeans_homogeneity_fc1'][iRun,0] = clusterResults[0]
    dicResults['kmeans_completeness_fc1'][iRun,0] = clusterResults[1]
    dicResults['kmeans_v_measure_fc1'][iRun,0] = clusterResults[2]
    clusterResultsPCA = clusterFeatures(fc1_featuresPCA,labelsAv, norm = "normal")
    dicResults['kmeans_homogeneity_pca_fc1'][iRun,0] = clusterResultsPCA[0]
    dicResults['kmeans_completeness_pca_fc1'][iRun,0] = clusterResultsPCA[1]
    dicResults['kmeans_v_measure_pca_fc1'][iRun,0] = clusterResultsPCA[2]
    if 'fc2_features' in locals():
        clusterResults = clusterFeatures(fc2_features,labelsAv, norm = "normal")
        dicResults['kmeans_homogeneity_fc2'][iRun,0] = clusterResults[0]
        dicResults['kmeans_completeness_fc2'][iRun,0] = clusterResults[1]
        dicResults['kmeans_v_measure_fc2'][iRun,0] = clusterResults[2]
        clusterResultsPCA = clusterFeatures(fc2_featuresPCA,labelsAv, norm = "normal")
        dicResults['kmeans_homogeneity_pca_fc2'][iRun,0] = clusterResultsPCA[0]
        dicResults['kmeans_completeness_pca_fc2'][iRun,0] = clusterResultsPCA[1]
        dicResults['kmeans_v_measure_pca_fc2'][iRun,0] = clusterResultsPCA[2]

    # GMM clustering
    clusterResults,_ = clusterGMMFeatures(fc1_features,labelsAv, norm = "normal")
    dicResults['GMM_homogeneity_fc1'][iRun,0] = clusterResults[0]
    dicResults['GMM_completeness_fc1'][iRun,0] = clusterResults[1]
    dicResults['GMM_v_measure_fc1'][iRun,0] = clusterResults[2]
    clusterResultsPCA,_ = clusterGMMFeatures(fc1_featuresPCA,labelsAv, norm = "normal")
    dicResults['GMM_homogeneity_pca_fc1'][iRun,0] = clusterResultsPCA[0]
    dicResults['GMM_completeness_pca_fc1'][iRun,0] = clusterResultsPCA[1]
    dicResults['GMM_v_measure_pca_fc1'][iRun,0] = clusterResultsPCA[2]
    if 'fc2_features' in locals():
        clusterResults,_ = clusterGMMFeatures(fc2_features,labelsAv, norm = "normal")
        dicResults['GMM_homogeneity_fc2'][iRun,0] = clusterResults[0]
        dicResults['GMM_completeness_fc2'][iRun,0] = clusterResults[1]
        dicResults['GMM_v_measure_fc2'][iRun,0] = clusterResults[2]
        clusterResultsPCA,_ = clusterGMMFeatures(fc2_featuresPCA,labelsAv, norm = "normal")
        dicResults['GMM_homogeneity_pca_fc2'][iRun,0] = clusterResultsPCA[0]
        dicResults['GMM_completeness_pca_fc2'][iRun,0] = clusterResultsPCA[1]
        dicResults['GMM_v_measure_pca_fc2'][iRun,0] = clusterResultsPCA[2]

    # Correlation with size
    dicResults['corrSize_fc1'][iRun,0] = correlateSize(fc1_features,scansAv,labelsAv)
    dicResults['corrSize_pca_fc1'][iRun,0] = correlateSize(fc1_featuresPCA,scansAv,labelsAv)
    if 'fc2_features' in locals():
        dicResults['corrSize_fc2'][iRun,0] = correlateSize(fc2_features,scansAv,labelsAv)
        dicResults['corrSize_pca_fc2'][iRun,0] = correlateSize(fc2_featuresPCA,scansAv,labelsAv)
    
    if is_tsne:
        fc1_features0=fc1_features
        fc1_features_noAv0=fc1_features_noAv

    if is_pcaVis:
        # Copy Pre-finetuning features
        fc1_featuresPCA0=fc1_featuresPCA
        fc1_featuresPCA_noAv0=fc1_featuresPCA_noAv
        
    # Accuracy before training:
    #print(model.metrics_names)
    if is_domainAdv:
        dicResults['acc'][iRun,0] = model.evaluate([x_train], [y_train, y_trainDomain], batch_size=batch_size)[3]
        dicResults['acc_domain'][iRun,0] = model.evaluate([x_train], [y_train, y_trainDomain], batch_size=batch_size)[4]
    else:
        dicResults['acc'][iRun,0] = model.evaluate([x_train], y_train, batch_size=batch_size)[1]

    # To change the loss weight values
    #K.set_value(lw1,1.)
    #K.set_value(lw2,0.)

    for iEp in range(totalEpochs):
        print('iEp:',iEp)
        if is_domainAdv:
            #if iEp>10:
            #    K.set_value(lw1,0)
            #    K.set_value(lw2,1)              
            #K.set_value(gr.hp_lambda,float(iEp)/totalEpochs)

            if is_dataAugm:
                reload(image)
                datagen = image.ImageDataGenerator(
                    #random_resolution=True,
                    width_shift_range=.1,
                    height_shift_range=.1,
                    horizontal_flip=True,
                    vertical_flip=True
                    )
                datagen.fit(x_train)
                hist = model.fit_generator(datagen.flow(x_train, [y_train, y_trainDomain], batch_size=batch_size,
                                    save_to_dir='results/augmented_images', 
                                    save_prefix='augm_im'),
                                    steps_per_epoch=len(x_train) / batch_size, epochs=1)
            else:
                hist = model.fit([x_train], [y_train, y_trainDomain], epochs=1, batch_size=batch_size)
        else:
            if is_dataAugm:
                reload(image)
                datagen = image.ImageDataGenerator(
                    #random_resolution=True,
                    width_shift_range=.1,
                    height_shift_range=.1,
                    horizontal_flip=True,
                    vertical_flip=True
                    )
                datagen.fit(x_train)
                hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size,
                                    save_to_dir='results/augmented_images', 
                                    save_prefix='augm_im'),
                                    steps_per_epoch=len(x_train) / batch_size, epochs=1)
            else:
                hist = model.fit(x_train, y_train, epochs=1, batch_size=batch_size)


        if network=='VGG':
            if is_addLayer:
                intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('fc3').output)
                fc1_features = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))
            else:
                intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('fc1').output,model.get_layer('fc2').output])
                [fc1_features,fc2_features] = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))
        elif network=='resnet50':
            if is_addLayer:
                intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('fc1').output])            
            else:
                intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('flatten_1').output])
            fc1_features = np.asarray(intermediate_layer_model.predict(x_all[idxTest]))
            fc1_featuresTrain = np.asarray(intermediate_layer_model.predict(x_all[idxTrain])) 
        
        # PCA before averaging
        # Normalize 
        fc1_features_noAv = normalize(fc1_features)
        
        # Remove NaN
        fc1_features_noAv = fc1_features_noAv[:,~np.isnan(fc1_features_noAv).any(axis=0)]
        fc1_featuresPCA_noAv = decomposition.PCA(n_components = nPCA).fit_transform(fc1_features_noAv)

        # Average within cart:
        fc1_features = cartridgeAverage(fc1_features,array_scans[idxTest],y_all[idxTest])
        if 'fc2_features' in locals():
            fc2_features = cartridgeAverage(fc2_features,array_scans[idxTest],y_all[idxTest])

        shapeFeat = fc1_features.shape
        fc1_features = np.reshape(fc1_features,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))
        if 'fc2_features' in locals():
            fc2_features = np.reshape(fc2_features,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))

        # Normalize 
        fc1_features = normalize(fc1_features)

        if 'fc2_features' in locals():
            av = np.mean(fc2_features,axis=0)
            stdv = np.std(fc2_features,axis=0)
            fc2_features = (fc2_features-av)/stdv

        # Remove NaN
        fc1_features = fc1_features[:,~np.isnan(fc1_features).any(axis=0)]
        if 'fc2_features' in locals():
            fc2_features = fc2_features[:,~np.isnan(fc2_features).any(axis=0)]

        # PCA
        fc1_featuresPCA = decomposition.PCA(n_components = nPCA).fit_transform(fc1_features) 
        if 'fc2_features' in locals():
            fc2_featuresPCA = decomposition.PCA(n_components = nPCA).fit_transform(fc2_features) 

        # Save the accuracy
        if is_domainAdv:
            dicResults['acc'][iRun,iEp+1] = hist.history['predictions_acc'][0]
            dicResults['acc_domain'][iRun,iEp+1] = hist.history['predictions2_acc'][0]
        else:
            dicResults['acc'][iRun,iEp+1] = hist.history['acc'][0]

        #Intraclass Correlation Coefficient (ICC)
        dicResults['icc_fc1'][iRun,iEp+1] = iccAverage(fc1_features,nTestLabels,nTestScans,type_icc)
        dicResults['icc_pca_fc1'][iRun,iEp+1] = iccAverage(fc1_featuresPCA,nTestLabels,nTestScans,type_icc)
        if 'fc2_features' in locals():
            dicResults['icc_fc2'][iRun,iEp+1] = iccAverage(fc2_features,nTestLabels,nTestScans,type_icc)
            dicResults['icc_pca_fc2'][iRun,iEp+1] = iccAverage(fc2_featuresPCA,nTestLabels,nTestScans,type_icc)

        # KMEANS clustering
        clusterResults = clusterFeatures(fc1_features,labelsAv, norm = "normal")
        dicResults['kmeans_homogeneity_fc1'][iRun,iEp+1] = clusterResults[0]
        dicResults['kmeans_completeness_fc1'][iRun,iEp+1] = clusterResults[1]
        dicResults['kmeans_v_measure_fc1'][iRun,iEp+1] = clusterResults[2]
        clusterResultsPCA = clusterFeatures(fc1_featuresPCA,labelsAv, norm = "normal")
        dicResults['kmeans_homogeneity_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[0]
        dicResults['kmeans_completeness_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[1]
        dicResults['kmeans_v_measure_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[2]
        if 'fc2_features' in locals():
            clusterResults = clusterFeatures(fc2_features,labelsAv, norm = "normal")
            dicResults['kmeans_homogeneity_fc2'][iRun,iEp+1] = clusterResults[0]
            dicResults['kmeans_completeness_fc2'][iRun,iEp+1] = clusterResults[1]
            dicResults['kmeans_v_measure_fc2'][iRun,iEp+1] = clusterResults[2]
            clusterResultsPCA = clusterFeatures(fc2_featuresPCA,labelsAv, norm = "normal")
            dicResults['kmeans_homogeneity_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[0]
            dicResults['kmeans_completeness_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[1]
            dicResults['kmeans_v_measure_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[2]
            
        # GMM clustering
        clusterResults,_ = clusterGMMFeatures(fc1_features,labelsAv, norm = "normal")
        dicResults['GMM_homogeneity_fc1'][iRun,iEp+1] = clusterResults[0]
        dicResults['GMM_completeness_fc1'][iRun,iEp+1] = clusterResults[1]
        dicResults['GMM_v_measure_fc1'][iRun,iEp+1] = clusterResults[2]
        clusterResultsPCA,_ = clusterGMMFeatures(fc1_featuresPCA,labelsAv, norm = "normal")
        dicResults['GMM_homogeneity_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[0]
        dicResults['GMM_completeness_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[1]
        dicResults['GMM_v_measure_pca_fc1'][iRun,iEp+1] = clusterResultsPCA[2]
        if 'fc2_features' in locals():
            clusterResults,_ = clusterGMMFeatures(fc2_features,labelsAv, norm = "normal")
            dicResults['GMM_homogeneity_fc2'][iRun,iEp+1] = clusterResults[0]
            dicResults['GMM_completeness_fc2'][iRun,iEp+1] = clusterResults[1]
            dicResults['GMM_v_measure_fc2'][iRun,iEp+1] = clusterResults[2]
            clusterResultsPCA,_ = clusterGMMFeatures(fc2_featuresPCA,labelsAv, norm = "normal")
            dicResults['GMM_homogeneity_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[0]
            dicResults['GMM_completeness_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[1]
            dicResults['GMM_v_measure_pca_fc2'][iRun,iEp+1] = clusterResultsPCA[2]
            
        # Correlation with size
        dicResults['corrSize_fc1'][iRun,iEp+1] = correlateSize(fc1_features,scansAv,labelsAv)
        dicResults['corrSize_pca_fc1'][iRun,iEp+1] = correlateSize(fc1_featuresPCA,scansAv,labelsAv)
        if 'fc2_features' in locals():
            dicResults['corrSize_fc2'][iRun,iEp+1] = correlateSize(fc2_features,scansAv,labelsAv)
            dicResults['corrSize_pca_fc2'][iRun,iEp+1] = correlateSize(fc2_featuresPCA,scansAv,labelsAv)
        if iEp%10==0:
            if is_tsne:
                # t-SNE to visualize entanglement
                fig = plt.figure(figsize=(30, 20))
                Y = TSNE(n_components=2).fit_transform(fc1_features0)
                ax = fig.add_subplot(2, 4, 1)
                # t-SNE  of test features (or train when testTrain) averaged within cart with labels colors
                plt.scatter(Y[:, 0], Y[:, 1], c=labelsAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)

                # t-SNE  of test features averaged within cart with scans colors
                ax = fig.add_subplot(2, 4, 2)
                plt.scatter(Y[:, 0], Y[:, 1], c=scansAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                # _all in the sense before averaging.  t-SNE of test features before averaging with labels colors
                Y = TSNE(n_components=2).fit_transform(fc1_features_noAv0)
                ax = fig.add_subplot(2, 4, 3)
                plt.scatter(Y[:, 0], Y[:, 1], c=y_all[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)
                # t-SNE of test features before averaging with scans colors
                ax = fig.add_subplot(2, 4, 4)
                plt.scatter(Y[:, 0], Y[:, 1], c=array_scans[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                Y = TSNE(n_components=2).fit_transform(fc1_features)
                ax = fig.add_subplot(2, 4, 5)
                plt.scatter(Y[:, 0], Y[:, 1], c=labelsAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)

                ax = fig.add_subplot(2, 4, 6)
                plt.scatter(Y[:, 0], Y[:, 1], c=scansAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                Y = TSNE(n_components=2).fit_transform(fc1_features_noAv)
                ax = fig.add_subplot(2, 4, 7)
                plt.scatter(Y[:, 0], Y[:, 1], c=y_all[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)

                ax = fig.add_subplot(2, 4, 8)
                plt.scatter(Y[:, 0], Y[:, 1], c=array_scans[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                ax.xaxis.set_major_formatter(NullFormatter())
                ax.yaxis.set_major_formatter(NullFormatter())
                plt.axis('tight')
                # Plots can be saved for each run, each epoch with:
                #fig.savefig('results/tsne/tsne_'+str(iRun)+'_'+str(iEp)+'.svg', bbox_inches='tight')
            if is_pcaVis:
                # PCA first components visualization
                fig = plt.figure(figsize=(30, 20))
                ax = fig.add_subplot(2, 4, 1)
                # PCA of test features (or train when testTrain) averaged within cart with labels colors
                plt.scatter(fc1_featuresPCA0[:, 1], fc1_featuresPCA0[:, 0], c=labelsAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)
                ax = fig.add_subplot(2, 4, 2)
                # PCA of test features averaged within cart with scans colors
                plt.scatter(fc1_featuresPCA0[:, 1], fc1_featuresPCA0[:, 0], c=scansAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                ax = fig.add_subplot(2, 4, 3)
                # _all in the sense before averaging.  PCA of test features before averaging with labels colors
                plt.scatter(fc1_featuresPCA_noAv0[:, 1], fc1_featuresPCA_noAv0[:, 0], c=y_all[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)
                ax = fig.add_subplot(2, 4, 4)
                #PCA of test features before averaging with scans colors
                plt.scatter(fc1_featuresPCA_noAv0[:, 1], fc1_featuresPCA_noAv0[:, 0], c=array_scans[idxTest], marker='x', linewidths=.2, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                ax = fig.add_subplot(2, 4, 5)
                plt.scatter(fc1_featuresPCA[:, 1], fc1_featuresPCA[:, 0], c=labelsAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)
                ax = fig.add_subplot(2, 4, 6)
                plt.scatter(fc1_featuresPCA[:, 1], fc1_featuresPCA[:, 0], c=scansAv, marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                ax = fig.add_subplot(2, 4, 7)
                plt.scatter(fc1_featuresPCA_noAv[:, 1], fc1_featuresPCA_noAv[:, 0], c=y_all[idxTest], marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=10)
                ax = fig.add_subplot(2, 4, 8)
                plt.scatter(fc1_featuresPCA_noAv[:, 1], fc1_featuresPCA_noAv[:, 0], c=array_scans[idxTest], marker='x', linewidths=.5, cmap=plt.cm.get_cmap("gist_ncar"), vmin=0, vmax=17)

                ax.xaxis.set_major_formatter(NullFormatter())
                ax.yaxis.set_major_formatter(NullFormatter())
                plt.axis('tight')
                # Plots can be saved for each run, each epoch with:
                #fig.savefig('results/pca/pca_'+str(iRun)+'_'+str(iEp)+'.svg', bbox_inches='tight')

                
if not os.path.exists('results'):
    os.makedirs('results')
    
# Save the dictionary of results as npy (if more stats are needed):
np.save('./results/dicResults.npy', dicResults)

# Average results across runs
dicAvResults = {}
for key in dicResults.keys():
    dicAvResults[key]= np.mean(dicResults[key],axis=0)
    
# Save results as csv file
csv_path = './results/results.csv'
with open(csv_path, 'w') as f:
    for key in sorted(dicAvResults.keys()):
        f.write("%s,%s\n"%(key,dicAvResults[key]))

print('Results saved in:',csv_path)



In [ ]:
dicAvResults